In [ ]:
# default_exp searcher

# Searcher

> Searchers are a one of the main block of web2dataset. They define the code to scrap the web for images

In [ ]:
# hide
from nbdev.showdoc import *

here is the abstract class for the Searcher 

In [ ]:
# export
import json
import os
from abc import ABC, abstractmethod
from typing import List, Optional

import jsons

from web2dataset.cleaner import IdentityCleaner, MetaDataCleaner
from web2dataset.document import Document

In [ ]:
# exports

#TODO : not use chdir
class Searcher(ABC):
    def __init__(
        self, query: str, n_item: int, cleaners: Optional[List[MetaDataCleaner]] = None
    ):
        self.query = query
        self.n_item = n_item
        self.documents: List[Document] = []
        self.cleaners = cleaners

    @abstractmethod
    def search(self):
        pass

    def clean(self):
        if self.cleaners is not None:
            for cleaner in self.cleaners:
                self.documents = cleaner.clean(self.documents)

    def save(self, path: str):
        """
        path: folder path
        """

        if not os.path.isdir(path):
            os.mkdir(path)
        os.chdir(path)

        if not os.path.isdir("metadata"):
            os.mkdir("metadata")
        os.chdir("metadata")

        for doc in self.documents:
            with open(f"{doc.uuid}.json", "w") as fp:
                json.dump(jsons.dump(doc), fp)

Example: simple searcher:
this class will be used of testing

In [ ]:
# export
class BasicSearcher(Searcher):
    def search(self):
        self.documents += [
            Document(
                origin="https://www.google.fr",
                image_url="https://raw.githubusercontent.com/fastai/fastai/master/nbs/images/puppy.jpg",
            )
            for _ in range(self.n_item)
        ]

let's define a searcher that will search random wallpaper on the web

In [ ]:
cleaners = [IdentityCleaner()]
searcher = BasicSearcher("wallpaper", 2, cleaners)
searcher.search()
searcher.clean()
assert len(searcher.documents) == searcher.n_item

we can then save the documents by giving a folder path to the method save_docs_to_file

In [ ]:
searcher.save("/tmp/my_search")

let's define the core error as well

In [ ]:
# export
class SearchError(ValueError):
    pass

# Google image searcher

> The Google searcher is the first and main searcher

In [ ]:
# export
from contextlib import contextmanager

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.common.by import By
from selenium.webdriver.remote.webelement import WebElement

first lets design a simple context manager to open and close the web driver automatically

In [ ]:
# export
@contextmanager
def _get_driver(debug=False):

    if debug:
        driver = webdriver.Chrome()
    else:
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        driver = webdriver.Chrome(options=chrome_options)
    try:
        yield driver
    finally:
        driver.quit()

In [ ]:
with _get_driver() as driver:
    pass

Then the google image searcher

In [ ]:
# export
class GoogleImageSearchError(SearchError):
    pass

In [ ]:
# export
class GoogleImageSearcher(Searcher):
    """
    query: str, it contains the query that is send to google search
    """

    def __init__(
        self, query: str, n_item: int, cleaners: Optional[List[MetaDataCleaner]] = None
    ):
        super().__init__(query, n_item, cleaners)

        self.google_image_url = self._create_url_from_query(self.query)

    def _create_url_from_query(self, query: str) -> str:
        if "+" in query:
            raise GoogleImageSearchError(
                " + should not be in the query because the whitespace are replaced by + so the meaning is different"
            )
        return f"https://www.google.com/search?q={query.replace(' ','+')}&source=lnms&tbm=isch"

    def _element_to_document(self, element):
        url = element.get_attribute("src")
        return Document(origin=self.query, image_url=url)

    def _find_images(self, driver: WebDriver) -> List[WebElement]:
        return driver.find_elements(By.CLASS_NAME, "rg_i")

    def _scrap_all_images_in_current_page(self, driver: WebDriver):
        elements = self._find_images(driver)
        self.elements = elements
        self.documents += [
            self._element_to_document(e)
            for i, e in enumerate(elements)
            if len(self.documents) + i < self.n_item
        ]

    def _scroll_to_next_page(self, driver: WebDriver):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    def search(self):
        with _get_driver() as driver:

            driver.get(self.google_image_url)

            _continue = True
            while _continue:
                self._scrap_all_images_in_current_page(driver)
                _continue = len(self.documents) < self.n_item

                if _continue:
                    self._scroll_to_next_page(driver)

In [ ]:
ggl_searcher = GoogleImageSearcher(
    "race bike",
    100,
)

In [ ]:
%%time
ggl_searcher.search()

CPU times: user 98.8 ms, sys: 25.2 ms, total: 124 ms
Wall time: 3.15 s


In [ ]:
assert len(ggl_searcher.documents) == ggl_searcher.n_item

In [ ]:
url = [doc.image_url for doc in ggl_searcher.documents]
len(url), len(set(url))

(100, 42)